In [ ]:
# 1. Necessary Libraries
!pip install transformers accelerate confluent-kafka torch torchvision facenet-pytorch opencv-python-headless numpy huggingface_hub

import cv2
import torch
import numpy as np
import json
import os
from huggingface_hub import login
from confluent_kafka import Consumer, Producer
from facenet_pytorch import MTCNN
from PIL import Image
from transformers import AutoModelForImageClassification, AutoImageProcessor
from google.colab.patches import cv2_imshow
from google.colab import drive
from huggingface_hub import login
import os

In [ ]:
# 2. Setup & Authentication
drive.mount("/content/drive", force_remount=True)
secrets_dir = '/content/drive/MyDrive/NeuroLens_Certs/'
config_file = os.path.join(secrets_dir, 'config.json')

with open(config_file, 'r') as f:
    secrets = json.load(f)

# Read token from environment variable
HF_TOKEN = os.getenv("HF_TOKEN")

if HF_TOKEN is None:
    raise RuntimeError("HF_TOKEN not found in environment variables")

login(token=HF_TOKEN)


MODEL_ID = "prithivMLmods/Deep-Fake-Detector-v2-Model"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 3. Load SOTA Model
try:
    print(f"🔄 Loading SOTA Deepfake Model: {MODEL_ID}...")
    processor = AutoImageProcessor.from_pretrained(MODEL_ID)
    vit_model = AutoModelForImageClassification.from_pretrained(MODEL_ID).to(device)
    vit_model.eval()
    print("✅ System Ready: High-Accuracy Mode Active.")
except Exception as e:
    print(f"❌ Connection Error: {e}")

# 4. MTCNN & Kafka Setup
mtcnn = MTCNN(image_size=224, margin=20, keep_all=False, device=device)


conf = {
    'bootstrap.servers': secrets['KAFKA_URI'],
    'security.protocol': 'SSL',
    'ssl.ca.location': os.path.join(secrets_dir, 'ca.pem'),
    'ssl.certificate.location': os.path.join(secrets_dir, 'service.cert'),
    'ssl.key.location': os.path.join(secrets_dir, 'service.key'),
    'group.id': 'neurolens-fast-group',
    'auto.offset.reset': 'latest',
    'enable.auto.commit': True,
    'fetch.min.bytes': 1,

    'session.timeout.ms': 45000,
}
consumer = Consumer(conf)
consumer.subscribe(['video-frames'])
result_producer = Producer(conf)

def send_result_to_kafka(score, label):
    data = {"score": round(score, 4), "label": label}
    result_producer.produce('detection-results', json.dumps(data).encode('utf-8'))
    result_producer.flush()

# Optimization Variables
scores_history = []
window_size = 10

print("🚀 NeuroLens AI Brain is LIVE. Processing Stream...")

# --- Main Loop ---
try:
    while True:
        msgs = consumer.consume(num_messages=10, timeout=0.1)

        if not msgs: continue

        msg = msgs[-1]

        if msg.error():
            continue

        frame_bytes = np.frombuffer(msg.value(), dtype=np.uint8)
        frame = cv2.imdecode(frame_bytes, cv2.IMREAD_COLOR)

        if frame is not None:
            img_rgb = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            face_img = mtcnn(img_rgb)

            if face_img is not None:
                # MTCNN Tensor to PIL
                face_np = face_img.permute(1, 2, 0).cpu().numpy()
                face_np = ((face_np + 1) * 127.5).astype(np.uint8)
                face_pil = Image.fromarray(face_np)

                with torch.no_grad():
                    inputs = processor(images=face_pil, return_tensors="pt").to(device)
                    outputs = vit_model(**inputs)
                    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

                    # prithivMLmods Indexing: 0 = Realism, 1 = Deepfake
                    raw_fake_score = probs[0][1].item()

                # Smoothing Logic
                scores_history.append(raw_fake_score)
                if len(scores_history) > window_size: scores_history.pop(0)
                avg_score = sum(scores_history) / len(scores_history)

                label = "FAKE" if avg_score > 0.5 else "REAL"
                send_result_to_kafka(avg_score, label)

                # UI Overlay
                color = (0, 0, 255) if label == "FAKE" else (0, 255, 0)
                cv2.putText(frame, f'{label}: {avg_score*100:.1f}%', (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

            cv2_imshow(frame)
except Exception as e:
    print(f"Runtime Error: {e}")
finally:
    consumer.close()